## Library and Install Depedencies

In [3]:
# Install Depedencies
!pip install transformers==4.38.2 datasets==2.18.0 accelerate==0.27.2 bitsandbytes==0.42.0 peft==0.9.0 torch langchain==0.1.13 langchain_community==0.0.29
!pip install --upgrade numpy pandas transformers datasets torch

  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: 

ERROR: Operation cancelled by user
^C


In [1]:
# For reuirement.txt
!pip freeze > requirements.txt

from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# Library
import numpy as np
import pandas as pd
import os

# Load Data dan Akses ke Drive
from google.colab import drive

# Data Processing and Model Building
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset, DatasetDict

## Load Data

In [4]:
# Mounting GDrive
drive.mount('/content/drive')

# Load dataset train dan validation
url_train = 'https://drive.google.com/uc?id=1sfxMl3R4bpJGiIPFibvdeqcg3AHyy9v-'
url_valid = 'https://drive.google.com/uc?id=1p6SVXXlEsZrg2QAgu34bdgsHHI0d6MOq'

data_train = pd.read_csv(url_train)
data_valid = pd.read_csv(url_valid)

Mounted at /content/drive


In [5]:
# Menampilkan 5 data pada data_train
data_train.head()

,short_question,short_answer,tags_label
0,Apa itu aplikasi BloodConnect?,BloodConnect adalah platform digital yang meng...,App_Info
1,Bagaimana cara mendaftar di BloodConnect?,Anda bisa mendaftar dengan mengunduh aplikasi ...,App_Info
2,Fitur apa saja yang ada di BloodConnect?,BloodConnect memiliki fitur pendaftaran pendon...,App_Info
3,Apakah aplikasi ini gratis?,"Ya, aplikasi BloodConnect dapat diunduh dan di...",App_Info
4,Dimana saya bisa mengunduh BloodConnect?,Aplikasi BloodConnect tersedia di Google Play ...,App_Info


In [6]:
# Menampilkan 5 data pada data_train
data_valid.head()

,short_question,short_answer,tags_label
0,Apakah BloodConnect bisa digunakan di luar neg...,Saat ini BloodConnect hanya beroperasi di Indo...,App_Info
1,Bagaimana cara melaporkan masalah teknis di ap...,Anda dapat melaporkan masalah teknis melalui f...,App_Info
2,Apakah ada biaya langganan untuk BloodConnect?,"Tidak, aplikasi BloodConnect dapat diunduh dan...",App_Info
3,Bisakah saya melihat jadwal donor darah yang a...,"Ya, Anda bisa melihat jadwal dan lokasi event ...",App_Info
4,Bagaimana cara kerja fitur notifikasi darurat?,Fitur notifikasi darurat akan mengirimkan peri...,App_Info


In [ ]:
# Menggabungkan Data
#data = pd.concat([data_train, data_valid], ignore_index = True)
#data.to_csv("Medical.csv", index = False)
#data.head()

## Data Prepocessing

In [7]:
# Konversi Pandas DataFrame ke Hugging Face Dataset
hf_train = Dataset.from_pandas(data_train)
hf_valid = Dataset.from_pandas(data_valid)

# Cek struktur dataset HF yang baru
print("\nHugging Face Train Dataset Columns:", hf_train.column_names)
print("Hugging Face Valid Dataset Columns:", hf_valid.column_names)


Hugging Face Train Dataset Columns: ['short_question', 'short_answer', 'tags_label']
Hugging Face Valid Dataset Columns: ['short_question', 'short_answer', 'tags_label']


In [8]:
# Inisialisasi Tokenizer
model_name = "EleutherAI/pythia-160m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Tokenizer pad_token set to: {tokenizer.pad_token}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenizer pad_token set to: <|endoftext|>


In [9]:
# Fungsi agar data bisa digunakan oleh model
def preprocess_function_causal_lm(examples):
    # Menggabungkan QnA ke dalam format prompt dan menambahkan tokenizer.eos_token (End-Of-Sequence) di akhir answer
    texts = [
        f"### Question:\n{q.strip()}\n\n### Answer:\n{a.strip()}{tokenizer.eos_token}"
        for q, a in zip(examples['short_question'], examples['short_answer'])
    ]
    # Melakukan tokenisasi
    model_inputs = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()

    return model_inputs

In [10]:
# Melakukan preprocessing ke dataset train
print("\nApplying preprocessing to train dataset...")
tokenized_train_dataset = hf_train.map(
    preprocess_function_causal_lm,
    batched=True,
    remove_columns=hf_train.column_names
)

# Melakukan preprocessing ke dataset valid
print("\nApplying preprocessing to validation dataset...")
tokenized_valid_dataset = hf_valid.map(
    preprocess_function_causal_lm,
    batched=True,
    remove_columns=hf_valid.column_names
)


Applying preprocessing to train dataset...


Map:   0%|          | 0/1166 [00:00<?, ? examples/s]


Applying preprocessing to validation dataset...


Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

In [11]:
# Gabungkan dataset train dan valid ke dalam DatasetDict (Format Hugging Face Trainer)
tokenized_dataset = DatasetDict({
    "train": tokenized_train_dataset,
    "validation": tokenized_valid_dataset,
})

print("\nFinal Tokenized Dataset Structure:")
print(tokenized_dataset)
print("Example of tokenized train data entry:")
print(tokenized_dataset["train"][0])


Final Tokenized Dataset Structure:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1166
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1108
    })
})
Example of tokenized train data entry:
{'input_ids': [4118, 19782, 27, 187, 34, 4904, 352, 86, 247, 446, 1479, 9720, 14169, 27642, 32, 187, 187, 4118, 37741, 27, 187, 30456, 27642, 519, 267, 1240, 5147, 5865, 30966, 278, 1205, 13897, 284, 38522, 2242, 35243, 14773, 13681, 1240, 16447, 47415, 28954, 13681, 1240, 4706, 4595, 1524, 14, 2606, 13, 1167, 438, 1240, 17369, 268, 2083, 6656, 32752, 251, 263, 16447, 4151, 9720, 331, 536, 13681, 1240, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Base Directory

In [12]:
# Menyimpan semua proses build model ke drive
base_dir = "/content/drive/MyDrive/Capstone/best_model"

## Build Model

In [13]:
# Muat model untuk Causal Language Modeling
model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

In [14]:
# Konfigurasi argumen pelatihan
training_args = TrainingArguments(
    output_dir=f"{base_dir}/results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir=f"{base_dir}/logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=False,
    evaluation_strategy="epoch",
    report_to="tensorboard",
)

In [15]:
# Data Collator untuk menyiapkan batch data
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

## Training and Testing

In [16]:
# Inisialisasi Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"], # Split 'validation'
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [17]:
# Mulai pelatihan
print("\nStarting model training...")
trainer.train()
print("\nTraining finished!")


Starting model training...


Epoch,Training Loss,Validation Loss
1,0.347300,0.593611
2,0.403900,0.453596
3,0.183800,0.387438
4,0.346000,0.374668
5,0.171500,0.353259
6,0.249400,0.360051
7,0.138900,0.366089
8,0.109100,0.361414
9,0.083700,0.389366
10,0.070400,0.381510



Training finished!


## Evaluation

In [22]:
# Melihat metric akurasi model
evaluation_results = trainer.evaluate()
print("\nHasil Evaluasi Akhir:")
print(evaluation_results)


Hasil Evaluasi Akhir:
{'eval_loss': 0.38150960206985474, 'eval_runtime': 55.6301, 'eval_samples_per_second': 19.917, 'eval_steps_per_second': 4.979, 'epoch': 10.0}


## Save Model

In [18]:
# Menyimpan model
output_model_dir = "./fine-tuned-chatbot-model"
trainer.save_model(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print(f"\nFine-tuned model saved to {output_model_dir}")


Fine-tuned model saved to ./fine-tuned-chatbot-model


In [21]:
# Menyimpan hasil di drive
best_model = f"{output_model_dir}/final_model"

# Menyimpan model dan tokenizer ke Google Drive
print(f"\nMenyimpan model ke {base_dir}...")
trainer.save_model(best_model)
tokenizer.save_pretrained(best_model)
print(f"\nModel fine-tuned berhasil disimpan ke: {base_dir}")


Menyimpan model ke /content/drive/MyDrive/Capstone/best_model...

Model fine-tuned berhasil disimpan ke: /content/drive/MyDrive/Capstone/best_model
